In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [3]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [4]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
# selected_destinations = ["9FF55", "482E9", "9D883"]

'\nThese destinations are selected based on their query pool sizes, and how representative\nthey are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)\n\n9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),\n482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),\n9D883ED995259EA3E715122397FE0EF9 has query pool of size 2\n'

In [5]:

# def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.assign(
#         destination=df.apply(
#             lambda row: row.destination[0:5],
#             axis=1,
#         )
#     )
#     global selected_destinations
#     return df[df.destination.isin(selected_destinations)]


In [6]:
# unaugmented path
# path = "ray/criteo/bias_varying_epoch_size"

In [7]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [8]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [9]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [10]:
# u.save_data(path)

In [11]:
# augmented impressions path
# path = "ray/criteo/augmented_bias_varying_epoch_size"

In [12]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [13]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [14]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [15]:
# augmented convesrions path
path = "ray/criteo/augmented_conversions_bias_varying_epoch_size"

In [16]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [17]:
u.save_data(path)

In [18]:
df = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions.csv")
df.head()

,partner_id,user_id,filter,product_category3,count,conversion_timestamp,epsilon,aggregatable_cap_value,key
0,C13DEAF04388B219AC04416DF3DA0ED8,4CA3BDF6304103161BEFC3DEAE5D0C2A,NaN,FE31DD5AD3E857EBD4BA2C4FDF732ED4,1.0,1596439563,0.757429,5,0
1,5DC3ED9F13993DF6471EBE25BD88BD24,F0D52A53241B145050B5ED8C55DD4B5B,NaN,8A30BF8BD2B3E9D44D2938D42C387D2E,1.0,1596439718,0.740381,5,0
2,6BFFDE668C5BC3938C875D3D0940E699,DC248B6867334C450AFDE6853A599E1E,NaN,F6BFE2644C2435F720C79B3EE10D9E42,1.0,1596439787,0.511686,5,0
3,9D883ED995259EA3E715122397FE0EF9,304E5E4A0F9F200B5A1CE22238AE1DE3,NaN,844A857A93A322847D10FA733714E3A6,4.0,1596439926,0.511686,5,0
4,319A2412BDB0EF669733053640B80112,79178DA7BA27DA4E64C4841D14DE747C,NaN,FAE775DB79B7E40FB3E581B2755783A4,1.0,1596439969,0.511686,5,8


In [19]:
true_output = df.groupby(["partner_id", "key"])["count"].sum()

In [20]:
print(true_output.shape, true_output[true_output > 0].shape)

(921,) (921,)


In [21]:
impressions = pd.read_csv("../data/criteo/criteo_query_pool_impressions.csv")

In [22]:
columns = ["partner_id", "user_id", "product_category3", "count", "conversion_timestamp", "epsilon", "key"]
x = df.merge(impressions, how="left", on=["partner_id", "user_id"], suffixes=("", "impressions"))
synthetic_conversions = x.loc[x.click_timestamp.isna()][columns]
real_conversions = x.loc[~x.click_timestamp.isna()][columns]

synthetic_counts = synthetic_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()
real_counts = real_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()

In [23]:
# Looks like somehow, the reports can end up fully populated
# by synthetic conversions

completely_synthetic_queries = []
for key in synthetic_counts.keys():
    if key not in real_counts:
        completely_synthetic_queries.append(key)

completely_synthetic_queries

[]

In [24]:
rmsres = pd.read_csv("../logs/ray/criteo/augmented_conversions_bias_varying_epoch_size/rmsres.csv")
rmsres.head()

,destination,queries_rmsres,baseline,initial_budget,workload_size,num_days_per_epoch,num_days_attribution_window
0,01E57DAC509AE48685D6219DE7718BA8,0.017717,ipa,1,1000,7,30
1,01E57DAC509AE48685D6219DE7718BA8,NaN,ipa,1,1000,7,30
2,01E57DAC509AE48685D6219DE7718BA8,NaN,ipa,1,1000,7,30
3,0225D81C97298F6FD63C8868EE8E2228,0.016393,ipa,1,1000,7,30
4,0225D81C97298F6FD63C8868EE8E2228,0.016106,ipa,1,1000,7,30


In [25]:
# and the reports made up of only synthetic conversion records
# end up being the ones with na RMSREs.
for baseline, rdf in rmsres.loc[(rmsres.baseline != "ipa") & (rmsres.queries_rmsres.isna())].groupby(["baseline"]):
    print(f"__{baseline[0]}__")
    for partner, pdf in rdf.groupby(["destination"]):
        print(f"{partner[0]}:", pdf.shape[0])

In [26]:
for benchmark in range(1, 15):
    print(f"number of queries (by destination) with RMSREs >= {benchmark}")
    for baseline, rdf in rmsres.loc[(rmsres.baseline != "ipa") & (rmsres.queries_rmsres >= benchmark)].groupby(["baseline"]):
        print(f"\t{baseline[0]}")
        for partner, pdf in rdf.groupby(["destination"]):
            print(f"\t{partner[0]}:", pdf.shape[0])

number of queries (by destination) with RMSREs >= 1
number of queries (by destination) with RMSREs >= 2
number of queries (by destination) with RMSREs >= 3
number of queries (by destination) with RMSREs >= 4
number of queries (by destination) with RMSREs >= 5
number of queries (by destination) with RMSREs >= 6
number of queries (by destination) with RMSREs >= 7
number of queries (by destination) with RMSREs >= 8
number of queries (by destination) with RMSREs >= 9
number of queries (by destination) with RMSREs >= 10
number of queries (by destination) with RMSREs >= 11
number of queries (by destination) with RMSREs >= 12
number of queries (by destination) with RMSREs >= 13
number of queries (by destination) with RMSREs >= 14
